In [1]:
import pandas as pd
import numpy as np

from stats import SaccadeFeatures
from extractor import Extractor

In [2]:
data = pd.concat([pd.read_excel('itog_fix_1.xlsx'), pd.read_excel('itog_fix_2.xlsx')], axis=0)

In [3]:
x = 'norm_pos_x'
y = 'norm_pos_y'
t = 'start_timestamp'
dur = 'duration'
dis = 'dispersion'

sac_feats_stats = {
    'length': ['min', 'max'],
    'speed': ['mean', 'kurtosis']
}
sac_feats_stats_shift = {
    'length': ['max']
}

sf = SaccadeFeatures(x=x, y=y, t=t, duration=dur,
                     pk=['tekst', 'Participant'],
                     features_stats=sac_feats_stats,
                     shift_pk=['tekst'],
                     shift_features=sac_feats_stats_shift)

features = [
    SaccadeFeatures(features_stats=sac_feats_stats,
                    shift_features=sac_feats_stats_shift,
                    shift_pk=['tekst'])
]

extractor = Extractor(features=features, x=x, y=y, t=t, duration=dur, dispersion=dis,
                      pk=['Participant', 'tekst'],
                      return_df=True)

In [4]:
sf.fit_transform(data)

,sac_length_min,sac_length_max,sac_length_max_shift,sac_speed_mean,sac_speed_kurtosis
1_1,0.002657,0.433486,-0.008934,3.354343,5.209396
1_2,0.002650,0.442420,0.000000,2.640718,2.718729
2_1,0.003757,0.321837,-0.127119,2.790059,1.062294
2_2,0.003741,0.448955,0.000000,2.585629,4.223900
3_1,0.003663,0.365776,-0.207636,2.859377,3.861619
...,...,...,...,...,...
35_2,0.002170,1.269198,0.000000,3.458571,20.617815
36_1,0.002688,0.303197,-0.821007,2.834547,0.247980
36_2,0.002196,1.124204,0.000000,3.228725,23.727673
37_1,0.002885,0.317565,-0.454604,2.836139,0.640584


In [5]:
extractor.fit_transform(data)

,sac_length_min,sac_length_max,sac_length_max_shift,sac_speed_mean,sac_speed_kurtosis
1_1,0.002657,0.433486,-0.008934,3.354343,5.209396
1_2,0.003757,0.321837,-0.127119,2.790059,1.062294
1_3,0.003663,0.365776,-0.207636,2.859377,3.861619
1_4,0.000212,0.342315,-0.362239,2.753961,0.792342
1_5,0.002705,0.375434,-0.094030,2.631501,2.786345
...,...,...,...,...,...
2_33,0.001920,0.406550,0.000000,2.544267,-0.174805
2_34,0.007928,0.354386,-0.260736,2.776038,-0.412796
2_35,0.002170,1.269198,0.000000,3.458571,20.617815
2_36,0.002196,1.124204,0.000000,3.228725,23.727673
